# Read file as lines

In [1]:
#INPUT_FN = "input_test.txt" ; DEBUG = True #1928
INPUT_FN = "input.txt" ; DEBUG=False

with open(INPUT_FN) as f:
    lines = [l.rstrip() for l in f]

if lines[-1]=='':
    lines=lines[:,-1]

In [2]:
def printd(s):
    if DEBUG:
        print(s)

In [3]:
assert len(lines)==1

Maybe we can implement solution as an array or linked list

I will try to implement as an array, and use insert and remove elements

Elements are blocks

In [4]:
drive_list_blocks_orig = [] # (Id, size), ID can be None

id=0

for i,c in enumerate(lines[0]):
    if i%2==0:
        #file
        drive_list_blocks_orig.append( (id,int(c)) )
        id+=1
    else:
        drive_list_blocks_orig.append( (None, int(c)) )


In [5]:
drive_list_blocks_orig

[(0, 2),
 (None, 3),
 (1, 4),
 (None, 2),
 (2, 3),
 (None, 5),
 (3, 5),
 (None, 0),
 (4, 3),
 (None, 7),
 (5, 3),
 (None, 2),
 (6, 5),
 (None, 1),
 (7, 4),
 (None, 1),
 (8, 3),
 (None, 1),
 (9, 7),
 (None, 4),
 (10, 6),
 (None, 3),
 (11, 3),
 (None, 7),
 (12, 8),
 (None, 1),
 (13, 9),
 (None, 8),
 (14, 9),
 (None, 3),
 (15, 7),
 (None, 7),
 (16, 9),
 (None, 1),
 (17, 3),
 (None, 3),
 (18, 5),
 (None, 3),
 (19, 1),
 (None, 7),
 (20, 2),
 (None, 4),
 (21, 2),
 (None, 4),
 (22, 1),
 (None, 0),
 (23, 3),
 (None, 1),
 (24, 3),
 (None, 2),
 (25, 1),
 (None, 8),
 (26, 2),
 (None, 6),
 (27, 2),
 (None, 2),
 (28, 7),
 (None, 3),
 (29, 7),
 (None, 9),
 (30, 5),
 (None, 1),
 (31, 2),
 (None, 5),
 (32, 9),
 (None, 8),
 (33, 5),
 (None, 3),
 (34, 4),
 (None, 3),
 (35, 7),
 (None, 9),
 (36, 5),
 (None, 2),
 (37, 7),
 (None, 9),
 (38, 8),
 (None, 1),
 (39, 9),
 (None, 4),
 (40, 9),
 (None, 4),
 (41, 7),
 (None, 7),
 (42, 5),
 (None, 1),
 (43, 7),
 (None, 8),
 (44, 5),
 (None, 4),
 (45, 1),
 (None, 7)

In [6]:
max_id=id-1
max_id

9999

# Defrag

In [7]:
def find_index_of_id(id, drive):
    for i, el in enumerate(drive):
        if el[0]==id:
            return i
    return None

def find_index_of_free_space(drive, space_req_size):
    for i, el in enumerate(drive):
        id,size = el
        if id is None:
            if space_req_size <= size:
                return i
    return None

In [8]:
a= [0,1,2,3]
a.insert(1, "a")
a #inserts behind index

[0, 'a', 1, 2, 3]

In [9]:
drive_list_blocks_orig0 = drive_list_blocks_orig.copy() # copy

for id in range(max_id,-1,-1):
    index_of_id_to_move = find_index_of_id(id, drive_list_blocks_orig0)
    
    if index_of_id_to_move is None:
        printd(f"Could not find index of ID {id} ")
        break

    block_candidate = drive_list_blocks_orig0[index_of_id_to_move]
    size = block_candidate[1]
    print(f"{id=}, {index_of_id_to_move=}, {block_candidate=}")

    index_of_free_space = find_index_of_free_space(drive_list_blocks_orig0, size)

    if index_of_free_space is None:
        printd("No big enough location found")
        continue
    
    if index_of_free_space>index_of_id_to_move:
        printd("Free space at higher index. Don't move")
        continue

    id_dest, size_dest = drive_list_blocks_orig0[index_of_free_space]
    printd(f"Free space at index {index_of_free_space}, {id_dest=}, {size_dest=}")

    assert size_dest >= size

    #drive_list_blocks_orig0.pop(index_of_id_to_move)
    drive_list_blocks_orig0[index_of_id_to_move] = (None, size)
    
    drive_list_blocks_orig0[index_of_free_space] = block_candidate
    printd(drive_list_blocks_orig0)

    index_to_clean_check = index_of_id_to_move

    if size_dest>size:
        printd(f"{size_dest=}  >  {size=}")
        # insert remaining free space
        drive_list_blocks_orig0.insert(index_of_free_space+1, (None, size_dest-size) )
        printd(drive_list_blocks_orig0)
        index_to_clean_check+=1

    # clean up, sort consecutive empty spaces, check before and after change
    clean_before =False
    clean_after = False
    if drive_list_blocks_orig0[index_to_clean_check-1][0] is None:
            
            printd("clean up before")
            clean_before = True

    if index_to_clean_check< len(drive_list_blocks_orig0)-1:
        if drive_list_blocks_orig0[index_to_clean_check+1][0] is None:
            printd("clean up after")
            clean_after = True
    
    if clean_before or clean_after:
            id0 = index_to_clean_check
            total_size = drive_list_blocks_orig0[id0][1]
            ncleans = 0
            if clean_before:
                 total_size += drive_list_blocks_orig0[index_to_clean_check-1][1]
                 id0-=1
                 ncleans+=1
            if clean_after:
                 total_size += drive_list_blocks_orig0[index_to_clean_check+1][1]
                 ncleans+=1

            drive_list_blocks_orig0[id0] = (None, total_size)

            for i in range(ncleans):
                 drive_list_blocks_orig0.pop(id0+1)
        
            printd(drive_list_blocks_orig0)

id=9999, index_of_id_to_move=19998, block_candidate=(9999, 7)
id=9998, index_of_id_to_move=19996, block_candidate=(9998, 2)
id=9997, index_of_id_to_move=19995, block_candidate=(9997, 1)
id=9996, index_of_id_to_move=19993, block_candidate=(9996, 4)
id=9995, index_of_id_to_move=19992, block_candidate=(9995, 4)
id=9994, index_of_id_to_move=19990, block_candidate=(9994, 1)
id=9993, index_of_id_to_move=19989, block_candidate=(9993, 2)
id=9992, index_of_id_to_move=19987, block_candidate=(9992, 5)
id=9991, index_of_id_to_move=19986, block_candidate=(9991, 1)
id=9990, index_of_id_to_move=19984, block_candidate=(9990, 2)
id=9989, index_of_id_to_move=19983, block_candidate=(9989, 6)
id=9988, index_of_id_to_move=19982, block_candidate=(9988, 2)
id=9987, index_of_id_to_move=19980, block_candidate=(9987, 8)
id=9986, index_of_id_to_move=19978, block_candidate=(9986, 3)
id=9985, index_of_id_to_move=19976, block_candidate=(9985, 4)
id=9984, index_of_id_to_move=19975, block_candidate=(9984, 9)
id=9983,

In [10]:
drive_list_blocks_orig0

[(0, 2),
 (9998, 2),
 (9997, 1),
 (1, 4),
 (9994, 1),
 (9991, 1),
 (2, 3),
 (9996, 4),
 (9980, 1),
 (3, 5),
 (None, 0),
 (4, 3),
 (9999, 7),
 (5, 3),
 (9993, 2),
 (6, 5),
 (9974, 1),
 (7, 4),
 (9965, 1),
 (8, 3),
 (9963, 1),
 (9, 7),
 (9995, 4),
 (10, 6),
 (9990, 2),
 (9962, 1),
 (11, 3),
 (9992, 5),
 (9988, 2),
 (12, 8),
 (9957, 1),
 (13, 9),
 (9989, 6),
 (9981, 2),
 (14, 9),
 (9986, 3),
 (15, 7),
 (9985, 4),
 (9978, 3),
 (16, 9),
 (9955, 1),
 (17, 3),
 (9975, 2),
 (9940, 1),
 (18, 5),
 (9964, 3),
 (19, 1),
 (9982, 4),
 (9956, 3),
 (20, 2),
 (9976, 4),
 (21, 2),
 (9970, 4),
 (22, 1),
 (None, 0),
 (23, 3),
 (9939, 1),
 (24, 3),
 (9946, 2),
 (25, 1),
 (9987, 8),
 (26, 2),
 (9979, 5),
 (9937, 1),
 (27, 2),
 (9944, 2),
 (28, 7),
 (9954, 3),
 (29, 7),
 (9984, 9),
 (30, 5),
 (9930, 1),
 (31, 2),
 (9972, 5),
 (32, 9),
 (9977, 8),
 (33, 5),
 (9949, 3),
 (34, 4),
 (9942, 3),
 (35, 7),
 (9983, 9),
 (36, 5),
 (9932, 2),
 (37, 7),
 (9973, 7),
 (9929, 2),
 (38, 8),
 (9921, 1),
 (39, 9),
 (9926, 3)

# checksum

In [11]:
checksum = 0
id_count = 0
for i, size in drive_list_blocks_orig0:
    for j in range(size):
        if i is not None:
            checksum+= id_count*i
        id_count+=1

checksum

6636608781232

OK